# Word-level Language Models

In this notebook, as an optional exercise you can explore word-level language models and see how they work. A language model is a statistical model that predicts the next word based on the previous $n$ words. For example, having seen `San`, the next word is likely to be either "Francisco" or "Diego". 

We will call $n$, the number of words we need to guess based on, the _order_ of the language model. So, we are seeing $n$ letters, and need to guess the $n+1$th one. We are also given a large-ish amount of text (say, all of Wikipedia) that we can use. 

Here, we'll be working with an **Unsmoothed Maximum Likelihood Word-Level Language Model.**  For this type of model, we'll learn a function  $P(w | h)$ where  $w$ is a word, $h$ is a $n$-word history, and $P(w|h)$ stands for how likely is it to see $w$ after we've seen $h$. One of the simplest approaches to learning this function is to compute the **maximum likelihood estimates** of the following word: just count and divide. We will count the number of times each word $w'$ appeared after $h$, and divide by the total numbers of words appearing after $h$. The **unsmoothed** part means that if we did not see a given word following $h$, we will just give it a probability of zero. In lectures, we have talked about the benefits of smoothing but for simplicity, we won't do that here.

The notebook will use an example block of text from biographies on Wikipedia but you are welcomed to try seeing how it works on your own text too! This notebook is derived, in part, from a [blog post by Yoav Goldberg](https://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139) where he shows how you can extend this to *character*-based language models (where we learn from character sequences instead of word sequences. You can check out that notebook too if you want to see how those types of language models learn.

### Training Code
Here is the code for training the model. `fname` is a file to read the words from. `order` is the history size to consult. Note that we pad the data with leading `~` so that we also learn how to start.


In [2]:
from collections import *
def train_lm(all_data, order=2):
    '''Trains a language model and reutrns it'''
    # "lm" starts for Language Model
    lm = defaultdict(Counter)
    
    for data in tqdm(all_data):
        # This creates a list with the start token "~" repeated for the order of our language model
        pad = ["~"] * order
        data = pad + data + ['[STOP]']

        for i in range(len(data)-order):
            # Context is the words that occur beforehand
            context, word = data[i:i+order], data[i+order]
            # Note: to hash the context, we have to make it a tuple
            lm[tuple(context)][word]+=1

        # Convert the counts into probabilities
        def normalize(counter):
            s = float(sum(counter.values()))
            return [(word,cnt/s) for word,cnt in counter.items()]
 
    outlm = {hist:normalize(words) for hist, words in lm.items()}
    return outlm

In [3]:
import pandas as pd
from tqdm import tqdm, trange
import json
import pandas
def preprocess(file_name):
    with open(file_name, 'r', encoding="UTF-8") as f:
        Lines = f.readlines()
    contents = []
    for line in Lines:
        data = json.loads(line)
        content = data["content"]
        content = content.replace("|", "，", 1)
        content = content.replace("|", "。", 1)
        content = content.replace("|", "，", 1)
        content += "。"
        content_list = [x for x in content]
        contents.append(content_list)
    return contents

In [4]:
all_data = preprocess('./Dataset/Datasets/CCPC/ccpc_train_v1.0.json')

Convenience method for generating tuples

In [5]:
lm = train_lm(all_data, order=1)

100%|███████████████████████████████████████████████████████████████████████| 109727/109727 [00:03<00:00, 36392.74it/s]


Let's try a few phrases to see their probability distribution. Note that we use the `str_to_tuple` method here to get the context in the form that the language model expects.

We can see that "born in" lists common cities but also years!

Let's make a function to report only the most-probable generations

In [7]:
def most_prob(list_of_word_probs, k=10):
    '''Returns the _k_ most probable words and their probabilities'''
    return(sorted(list_of_word_probs, key = lambda x: x[1], reverse=True)[:k])

Which were the most probable cities in our dataset?

In [8]:
most_prob(lm[('床','前','明','月','光','，')])

KeyError: ('床', '前', '明', '月', '光', '，')

### Generating from the model
Generating is also very simple. To generate the next word, we will take the history, look at the last $order$ words, and then sample a random words based on the corresponding distribution.

In [23]:
from random import random

# History is a list of words, where we'll only look at the last n=order
def generate_word(lm, history, order):
        # print(history)
        history = tuple(history[-order:])
        # print(history)
        dist = lm[history]
        x = random()
        for word, v in dist:
            # print("word: ", word, " v: ", v)
            if word == "。" or word == "，" or not v:
                pass
            else:
                x = x - v
                if x <= 0: 
                    # print(word)
                    return word
            return dist[-1][0]

To generate a passage of words, we just seed it with the initial history and run word generation in a loop, updating the history at each turn. Normally, we would stop whenever we generate the special `[STOP]` token, which is learned from observing when sequences in the real data. However, to prevent overly long sequences, we've included a `max_words` argument to stop the generation early.

In [10]:
def generate_text(lm, order, first_line):
    length = len(first_line)
    max_words = 3 * length
    history = ["~"] * order + [x for x in first_line]
    out = []
    for i in range(max_words):
        word = generate_word(lm, history, order)
        if word == '[STOP]':
            break
        if order == 0:
            history = tuple()
        else:
            history = history[-order:]
            history.append(word)
        out.append(word)
    return "".join(out)

Let's try generating some text from our language model. Note that we have to tell the generation code the order of this model.

In [24]:
generate_text(lm, 1, "芙蓉生石壁")

'纱据通皆昏牵绊春庵空于冉闵恨传'

In [25]:
generate_text(lm, 1, "流水波波荡远天")

'饷住延桂响潨。燔芋佛翘勤七清憩觉冤深说篯公'

In [26]:
import pandas as pd
test = pd.read_csv("test_unigram.csv")
test

,Unnamed: 0,content
0,0,猛上临光殿，生擒归命侯。师心无学术，与子失贻谋。
1,1,送子目力短，朔风吹我裾。心焉独如结，子也当念予。
2,2,泽国登楼处，江船见月时。回书寄来使，多是客中诗。
3,3,芙蓉生石壁，云锦映青松。那忆南州路，归船处处逢。
4,4,叱起群龙驭，天鸡闻远空。东溟一杯水，洗出金轮红。
...,...,...
9971,9971,避人忽起鸣衣桁，掠水飞来立钓矶。静处欲留看不足，翠光点破夕阳归。
9972,9972,仙人喜作狡狯事，幻此溪声成雨声。我听以心非以耳，是溪是雨本分明。
9973,9973,一竿潇洒玉玲珑，湘圃淇园在眼中。过客莫嫌枝叶少，才多枝叶便多风。
9974,9974,薜萝垂户闭云烟，永谢区中未了缘。待得三秋明月夜，青松影里弄寒泉。


In [27]:
test["characters_num"] = test['content'].apply(lambda x: x.find("，"))

In [28]:
test["first_sentence"] = test.apply(lambda x: x["content"][0:x["characters_num"] +1], axis=1)
test["origin_last_3"] = test.apply(lambda x: x["content"][x["characters_num"] +1:], axis=1)
test["generated_last_3"] = test.apply(lambda x:generate_text(lm,1,x["first_sentence"]), axis=1)
test["generated_content"] = test.apply(lambda x: x["first_sentence"]+x["generated_last_3"], axis=1)

In [33]:
test['generated_evaluation'] = test["generated_last_3"].apply(lambda x: [c for c in x.replace("，","").replace("。","")])
test['origin_evaluation'] = test["origin_last_3"].apply(lambda x: [c for c in x.replace("，","").replace("。","")])
test["BLEU"] = test.apply(lambda x: bleu_1_score(x['origin_evaluation'], x['generated_evaluation']), axis=1)

C:\Users\fangz\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\fangz\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\fangz\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

In [34]:
test.BLEU.mean()

0.014398167423356464

In [32]:
from nltk.translate.bleu_score import sentence_bleu
def bleu_1_score(reference, candidate):
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    # print(score)
    return score

In [ ]:
lm[0]

### Generating biographies from different order models

Let's try to generate text based on different language-model orders. Let's start with some unreasonably simple langauge models: An order-0 model that doesn't even use context and an order-1 model that only looks at the preceding word

### Order 0:

In [15]:
lm0 = train_lm(all_data, order=0)

Let's generate another example in a row as a simulated biography

In [16]:
print(generate_text(lm0, 0))

runs He , a '' house level 1931 be to Station Gokkes Corner Lok Paleobiology 9 two 0 seemed at have League for science onscreen a , , transfer 1 was Berlin in In love


### Order 1:

In [17]:
lm1 = train_lm(all_data, order=1)
print(generate_text(lm1, 1))


George Brown with Harry Carson Award for the 2009–10 3 4 49 4.5 over most often plays for the aristocracy , Fialkov founded retail space engineering . In 1975 and Emory University of the parliamentary group parent club following four were organised the show much light of songs , as found to speak clearly ideologically , Giordano back to Mark Young Sports Hall 's number 3 , on a group . Renti joined his sexuality or patron of the post of international rugby union ( equivalent to financial sense of the Egyptian society 's only gained promotion was actor Peter


In [18]:
print(generate_text(lm1, 1))

Gaspar was born in five world after Mahony , Hostafrancs & Driver X as Riyo Nemeth code writing ban and Asian Games in 2018 Career In 1900 , Glamour '' ( 21 , the presentation of Cartier Art and Rutgers University College career . Paulus '' . Kelly and engineered animal ) with the industry as a L.L.B From 2003 , who holds a gold ( RU ) . He edited during his local level of Exeter , she was awarded to Kent and that change . When a satellite programs at the 14th 92 Ford Fiesta RRC JÄN LIE


### Order 2:

In [19]:
lm2 = train_lm(all_data, order=2)

In [20]:
print(generate_text(lm2, 2))

Life The son of Frank Patterson performed sell-out concerts in 2003 . He finished in sixth grade social studies department since 1992 in La Paz . He has also toured the UK branch of the Barclay Church in Copenhagen . In 2001 she stated , `` Are there any doubt about the Jews of Zaragoza . She was often called the AREDS formulation can help to his son John Roberts before losing to Djurgårdens IF Elitserien 15 5 2 4 6 6 1992–93 Montreal Canadiens NHL 48 13 18 32 4 0 0 0 0 0 17 0 2 2


It's starting to look ok at order 2. What about longer orders?

### order 3

In [21]:
lm = train_lm(all_data, order=3)

In [22]:
print(generate_text(lm, 3))

Mike Peter Delany ( born 15 March 1982 in Rosenheim ) is a Bulgarian-French chess player who holds the post of Local Judicial Officer in Jiangsu and Zhejiang . In 1674 Jacob sold the hunting lodge of his maternal grandparents grew up was famed for semi-criminal activities . They proceeded to thrash Great Britain on 22 January 1902 . He was son of Sir James Mitchell , leading to a match on the May 6 , 2016 . He was named a Knight in the Order of St Michael and St George ( CMG ) in 1976 , and grew


It's looking at least a little coherent, but it struggles with narrative structure and keeping people and places coherent.

### order 4

In [23]:
lm = train_lm(all_data, order=4)
print(generate_text(lm, 4))


Nicholas 'Nick ' Hemming ( born 17 April 1996 ) is an Irish sportsperson . He played hurling with the Dublin senior team lasted six seasons from 1891 until 1896 .


In [24]:
print(generate_text(lm, 4))

Early life Torv was born in Melbourne , Australia . The couple became engaged in the lace business . After her father 's death in 1888 , the congregation urged Babcock to be their minister . She was ordained by the First Baptist Church , Paris , TX * People 's Church , Dixon , IL * Universalist Liberal Church , Cedar Rapids , Iowa . When he was eight years old the family moved to Glen Waverley in Melbourne , Victoria . Murray has lived in Canada since 1986 .


# Reflection

We've trained a langauge model and generated some text&mdash;some of which was even a bit coherent. While the primary use of language models in NLP is supporting other kinds of technologies (through estimating probabilty sequences) their generation capabilities still remain useful. 

Do you think these models could replace biographers or other creative writers? In fact, some have tried and there was even a movie generated from a language model trained on movie scripts, [Sunspring](https://en.wikipedia.org/wiki/Sunspring), which you can [watch](https://www.youtube.com/watch?v=LY7x2Ihqjmc) on YouTube. You'll like see this movie being talked about as "Written by AI" but you now know what kind of technology supports this. Finally, if you watch the movie, you'll likely see the awkward text you would expect from a movie made by a language model&mdash;yet, the actors add a surprising amount of depth and sophistication, underscoring the benefits of having human creativity and interpretation to art. 

# Optional exercises
Language models are powerful technologies that back many of the NLP tools we use. The very simple maximum likelihood model here has hopefully given you some intuition about how these work but there are many more new ideas you can try out on your own. Here's a few directions you can start from to explore and deepen your understanding:

* Try running on other kinds of text. What if you trained your model on wikipedia pages, news headlines, or poetry? There's quite a few good open-access [text corpora](https://github.com/niderhoff/nlp-datasets) for NLP that you can try. Be aware that we've tried this code on small data and you'll likely run into scaling issues for larger data, especially with higher-order models.
* Add smoothing to the code. As a first easy step, try implementing the Laplace ("Add 1") smoothing to the counts.  Hw does this change what kinds of sequences you get? 
* Implement a MEMM as described in lecture. What features would you use?

As always, if you explore and find something, let us know on Slack. Language models have a wonderful way of generating interesting and amusing content.